## OBJV:
- test the session for pydap and retrieve multiple files from openDAP

In [30]:
## libraries for data access and pre-processing
## preprocessing: retireve data from netCDF and convert to pandas columns
import netCDF4 # packages to open 'netcdf' file
import numpy as np # numpy and pandas packages to pre-process the dataset
import pandas as pd

# for visualization 
import matplotlib.pyplot as plt # to create plots and graphs
from mpl_toolkits.basemap import Basemap # to create geo-spatial map, requires dependencies installation
import plotly.express as px
import matplotlib.patches as mpatches
import matplotlib as mpl
import matplotlib.animation as animation
from matplotlib.collections import PatchCollection
import time
import matplotlib

# to load webcontent and retrieve data from link
from IPython.display import display, HTML
from IPython import display
from datetime import datetime

In [31]:
# WEB SCRAP the contents
import re
# to grab data from entire year, month
from bs4 import BeautifulSoup # 

In [32]:
# Data ACCESS from OPENDAD source
## Libraries
from urllib import request, parse
import getpass
import netrc
import os
import requests
import matplotlib.image as mpimg
import os
import time
from netCDF4 import Dataset

# pydap library to open session
from pydap import client
from pydap.cas.urs import setup_session
from pydap.client import open_url

import json

In [33]:
def get_session(url, file_name):
    """
    Creating a session with url and filename in openDap for data retrieval
    https://oco2.gesdisc.eosdis.nasa.gov/opendap/
    """
    try:
        login_credentials= 'uat.urs.earthdata.nasa.gov'
        username, _, password = netrc.netrc().authenticators(login_credentials)
    except (FileNotFoundError, TypeError):
        # FileNotFound = There's no .netrc file
        # TypeError = The endpoint isn't in the netrc file, causing the above to try unpacking None
        print('\n*******************************************\n')
        print('Please provide your Earthdata Login credentials to allow data access\n')
        print('Your credentials will only be passed to %s and will not be exposed in Jupyter' % (url))
        print('\n')
        username = input('Username: ')
        password = getpass.getpass()
        print('\n*******************************************\n')
        
    # pydap session
    session = setup_session(username, password, check_url= url + file_name)
    
    # using the session to get access the data
    return session

## NOTE:
1. First, retrieve the contents from the specified year, version of files
2. Get the first file from the list

### Data Source:
- main_url='https://oco2.gesdisc.eosdis.nasa.gov/opendap'

#### OCO content
- `OCO2_L2_Lite_FP`
- `OCO3_L2_Lite_FP`

### OCO3, openDAP source
- Example;
- https://oco2.gesdisc.eosdis.nasa.gov/opendap
- /OCO3_L2_Lite_FP.10.4r
- /2020
- /oco3_LtCO2_200106_B10400Br_220317235330s.nc4.html

In [34]:
%%time
print("OCO3 -> 10 vr.\n")
## INPUT for specified year, version of file
year= input("Enter the Year: ")
month= input("Enter the Month: ")
ver_= input("Enter the version: ")

## Select OCO2 content
#lite_file= '/OCO2_L2_Lite_FP.'+ str(ver_)+ 'r' +'/'+ year

## OCO3
lite_file= '/OCO3_L2_Lite_FP.'+ str(ver_)+ '.4r' +'/'+ year 

########################################################################
# GET THE files from OpenDap source. Open the netcdf file and store in dataframe
# Beautiful soup to retrieve all data for the MONTH
# EXAMPLE: in this test, we will use SINGLE file of the month
########################################################################
main_url='https://oco2.gesdisc.eosdis.nasa.gov/opendap'
content= '/contents.html'

s= requests.get(main_url + lite_file + content)

# Scrap the content by specified URL
soup= BeautifulSoup(s.content, 'html.parser')

list_files=[]
# html_links= soup.select('a[href$=".nc4.html"]')

html_links= soup.select('a[href$=".nc4.html"]')

for link in html_links:
    list_files.append(link['href'])

## pre-process the filenames; strings, CLEAN the files
# removing last strings '.html' to download the files from PYDAP library to match file names
files_oco2= [f[:-5] for f in list_files]

# total_files= ['opendap'+lite_file+'/'+ f for f in files_oco2[:3]]
total_files= [lite_file+'/'+ f for f in files_oco2]

## get alternate row files; duplicate ROWS on html LINKS
total_files= total_files[::2]

OCO3 -> 10 vr.

Enter the Year: 2021
Enter the Month: 08
Enter the version: 10
Wall time: 15 s


# GET session TOKEN from the request
- using the same `session` token to request multiple files

In [36]:
url=main_url

In [37]:
session= get_session(url, total_files[0])


*******************************************

Please provide your Earthdata Login credentials to allow data access

Your credentials will only be passed to https://oco2.gesdisc.eosdis.nasa.gov/opendap and will not be exposed in Jupyter


Username: sagarlimbu
········

*******************************************



In [38]:
len(total_files)

338

## Get files for SPECIFIED months only
- '/OCO3_L2_Lite_FP.10.4r/2020/oco3_LtCO2_20'
- Filter the files by the above filenames to get specific month files

In [39]:
%%time
total_= [f for f in total_files if f.startswith(total_files[0][:41]+month) ]
#total_= total_files

Wall time: 0 ns


In [40]:
len(total_)

25

### Following function;
- Performs iterative process to access and retrieve multiple files
- Using the same SESSION token generated from previous request to `OPENDAP` website

### Data Access to Multiple files
- takes in pydap data and appends all the variables together

In [ ]:
%%time
# session.post

## list of pydap variables
xco2=[]
long_vertices=[]
lat_vertices=[]
sounding_id=[]
qual_flag=[]

if session:
#     print("alive")
    for j in range(0, len(total_)):
        pydap_df= open_url(main_url + total_[j], session=session)
        print(pydap_df)
        
        #################################################
        # collect the data
        xco2.append(np.array(pydap_df["xco2"][:]))
        long_vertices.append(pydap_df["vertex_longitude"][:])
        lat_vertices.append(pydap_df["vertex_latitude"][:])
        qual_flag.append(pydap_df["xco2_quality_flag"][:])
#        sounding_id.append(pydap_df["date"])        

        print('***********************/n')
else:
    print("request new ssion")

<DatasetType with children 'xco2_apriori', 'file_index', 'xco2_qf_simple_bitflag', 'pressure_levels', 'xco2', 'time', 'pressure_weight', 'Preprocessors_h2o_ratio', 'Preprocessors_xco2_weak_idp', 'Preprocessors_max_declocking_o2a', 'Preprocessors_xco2_strong_idp', 'Preprocessors_dp_abp', 'Preprocessors_co2_ratio', 'Preprocessors_csstd_ratio_wco2', 'solar_zenith_angle', 'longitude', 'xco2_qf_bitflag', 'latitude', 'sensor_zenith_angle', 'Meteorology_psurf_apriori_o2a', 'Meteorology_psurf_apriori_wco2', 'Meteorology_psurf_apriori_sco2', 'Meteorology_windspeed_u_met', 'Meteorology_windspeed_v_met', 'xco2_quality_flag', 'xco2_averaging_kernel', 'date', 'Retrieval_dp_o2a', 'Retrieval_dust_height', 'Retrieval_aod_water', 'Retrieval_s32', 'Retrieval_chi2_sco2', 'Retrieval_aod_dust', 'Retrieval_albedo_slope_wco2', 'Retrieval_aod_bc', 'Retrieval_aod_strataer', 'Retrieval_eof2_2_rel', 'Retrieval_aod_seasalt', 'Retrieval_rms_rel_o2a', 'Retrieval_windspeed_apriori', 'Retrieval_albedo_slope_sco2', 'R

***********************/n
<DatasetType with children 'xco2_apriori', 'file_index', 'xco2_qf_simple_bitflag', 'pressure_levels', 'xco2', 'time', 'pressure_weight', 'Preprocessors_h2o_ratio', 'Preprocessors_xco2_weak_idp', 'Preprocessors_max_declocking_o2a', 'Preprocessors_xco2_strong_idp', 'Preprocessors_co2_ratio', 'Preprocessors_csstd_ratio_wco2', 'Preprocessors_dp_abp', 'solar_zenith_angle', 'longitude', 'xco2_qf_bitflag', 'latitude', 'sensor_zenith_angle', 'Meteorology_psurf_apriori_o2a', 'Meteorology_psurf_apriori_wco2', 'Meteorology_psurf_apriori_sco2', 'Meteorology_windspeed_u_met', 'Meteorology_windspeed_v_met', 'xco2_quality_flag', 'xco2_averaging_kernel', 'date', 'Retrieval_dp_o2a', 'Retrieval_dust_height', 'Retrieval_aod_water', 'Retrieval_s32', 'Retrieval_chi2_sco2', 'Retrieval_aod_dust', 'Retrieval_albedo_slope_wco2', 'Retrieval_aod_bc', 'Retrieval_aod_strataer', 'Retrieval_eof2_2_rel', 'Retrieval_aod_seasalt', 'Retrieval_rms_rel_o2a', 'Retrieval_windspeed_apriori', 'Retrie

## Pre-process the data
1. Pandas dataframe: filter the data by quality-flag-> 0
2. Dstack the vertices from CORNER points
3. Create Polygon PATCHES from matplotlib

In [ ]:
%%time
##################################################
# 1. Create pandas dataframe to filter the values
df_oco3= pd.DataFrame()
df_oco3["Xco2"]= np.array(np.concatenate(xco2)).ravel() 
df_oco3["Longitude_vertices"]= list(np.array(np.concatenate(long_vertices)))
df_oco3["Latitude_vertices"]= list(np.array(np.concatenate(lat_vertices)))
df_oco3["quality_flag"]= np.array(np.concatenate(qual_flag)).ravel()

##################################################
# Filter the DATA by quality flag-> 0
df_oco3= df_oco3[df_oco3["quality_flag"]== 0]
df_oco3=df_oco3.reset_index()
df_oco3= df_oco3.drop(columns=["index"])

#################################################
# 2. DSTACK the vertices of corner points together
flat_vertices=[]
for j in range(0, len(df_test)):
    flat_vertices.append(np.dstack([df_oco3["Longitude_vertices"].iloc[j], df_oco3["Latitude_vertices"].iloc[j] ]))

##################################################
# 3. GET patches from the vertices
## unpack the values from the list
unpack_vertices= [elem for sublist in flat_vertices for elem in sublist]
patches_total= [mpatches.Polygon(row) for row in unpack_vertices]

##################################################
## XCO2 values
xco2_total= df_oco3["Xco2"][:]

## Garbage Collect, release memory
del df_oco3
gc.collect()

# PLOT with basemap using the vertices
- NOTE: uses the entire data without any filtering on QUALITY_FLAG
- next work on ploting data with filtering values by quality flag-> 0

In [ ]:
%matplotlib notebook

In [ ]:
%%time
# ## XCO2 values
# xco2_total= xco2_list

## plot to graph using basemap
fig= plt.figure(111)
ax= fig.add_subplot(111)

## SET the min and max boundaries for XCO2
xco2_min_= 410
xco2_max_= 420

# world map: 
# set the boundaries
llcrnrlon, urcrnrlon, llcrnrlat, urcrnrlat= -180, 90, -90, 90 


# ###worldmap (FULL_OUTER_MAP)

m= Basemap(projection= 'cyl',
        llcrnrlon= llcrnrlon,
        urcrnrlon= urcrnrlon,
        llcrnrlat= llcrnrlat,
        urcrnrlat= urcrnrlat,   
        resolution='l', 
        epsg= 4269
        )

## DARK theme map
m.fillcontinents(color='#191919',lake_color='#000000') # dark grey land, black lakes
m.drawmapboundary(fill_color='#000000') 


### ArcGIS map
# m.arcgisimage(server='http://server.arcgisonline.com/ArcGIS', 
#               #service='World_Shaded_Relief', 
#               service='World_Imagery',
#               xpixels=2000, ypixels=None, dpi= 2000, verbose=False)


cmap= plt.get_cmap('viridis')
colors= cmap(xco2_total)

## normalize
norm= matplotlib.colors.Normalize(vmin= xco2_min_,
                                 vmax= xco2_max_)

## PATCHES object passed to this function
## patch collection and plt show
p= PatchCollection(patches_total,
                   cmap= matplotlib.cm.viridis, 
                   #alpha= 0.95, 
                  # linewidths= 4
                   edgecolor='none',
                   norm= norm
                  )

# set color range from XCO2
#p.set_color(colors)
p.set_array(xco2_total)
#p.set_clim(np.min(xco2), np.max(xco2))

plt.gcf().set_size_inches(10, 10)
ax.add_collection(p)

## COLOR bar
cmap = mpl.cm.viridis
norm = mpl.colors.Normalize(vmin=xco2_min_, vmax=xco2_max_)
cbar= plt.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
                   orientation='horizontal', label='OCO2, XCO2 ppm\nMonths: ')
#                   + str(months_sel))
#    plt.savefig('dark'+str(year)+'_'+str(months_sel)+"_oco2_oco3_comb.jpg", dpi= 3500, bbox_inches='tight', pad_inches= 2)
plt.show();